In [1]:
import os

Variaveis globais necessárias para funçoes

In [2]:
datasetTxt = "Datasets/Dataset-ClassTT_01_tiny_File.txt"


Parse do dataset de horários, retornando um dicionário com todas as seções.

    Retorna:
        dict: Dicionário com as seguintes chaves:
            - 'head': dict com configurações gerais
            - 'cc': dict {turma: [cursos]}
            - 'olw': list de cursos com apenas 1 aula/semana
            - 'dsd': dict {professor: [cursos]}
            - 'tr': dict {professor: [slots indisponíveis]}
            - 'rr': dict {curso: sala}
            - 'oc': dict {curso: índice_aula_online}

In [3]:
def parse_dataset(datasetTxt):
    lines = datasetTxt.strip().split('\n')
    data = {
        'head': {},
        'cc': {},
        'olw': [],
        'dsd': {},
        'tr': {},
        'rr': {},
        'oc': {}
    }

    current_section = None

    for line in lines:
        line = line.strip()

        # Ignorar linhas vazias
        if not line:
            continue

        # Detectar cabeçalhos de seção
        if line.startswith('#'):
            section_name = line.split()[0][1:]  # Remove o '#'
            current_section = section_name
            continue

        # Ignorar comentários
        if line.startswith('—') or line.startswith('--'):
            continue

        # Processar dados baseado na seção atual
        if current_section == 'cc':
            parts = line.split()
            if parts:
                class_id = parts[0]
                courses = parts[1:]
                data['cc'][class_id] = courses

        elif current_section == 'olw':
            # Esta seção parece estar vazia no exemplo
            if line.strip():
                data['olw'].append(line.strip())

        elif current_section == 'dsd':
            parts = line.split()
            if parts:
                teacher = parts[0]
                courses = parts[1:]
                data['dsd'][teacher] = courses

        elif current_section == 'tr':
            parts = line.split()
            if parts:
                teacher = parts[0]
                slots = [int(s) for s in parts[1:]]
                data['tr'][teacher] = slots

        elif current_section == 'rr':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                room = parts[1]
                data['rr'][course] = room

        elif current_section == 'oc':
            parts = line.split()
            if len(parts) >= 2:
                course = parts[0]
                week_index = int(parts[1])
                data['oc'][course] = week_index

    return data

Lê o ficheiro do dataset.

In [4]:
def load_dataset_from_file(datasetTxt):
    try:
        with open(datasetTxt, 'r', encoding='utf-8') as f:
            dataset_text = f.read()
        return parse_dataset(dataset_text)
    except FileNotFoundError:
        print(f"Erro: Ficheiro '{datasetTxt}' não encontrado!")
        return None
    except Exception as e:
        print(f"Erro ao ler ficheiro: {e}")
        return None

## Testes

In [5]:

# Carregar o dataset
dados = load_dataset_from_file(datasetTxt)

# Verificar se carregou
if dados is None:
    print("❌ Falha ao carregar o dataset!")
else:
    print("✅ Dataset carregado com sucesso!\n")

    # DEBUG: Ver o conteúdo de cada seção
    print("=" * 50)
    print("📊 CONTEÚDO PARSEADO:")
    print("=" * 50)

    print("\n🎓 HEAD (Configurações):")
    print(dados['head'])

    print("\n📚 CC (Classes e Cursos):")
    for turma, cursos in dados['cc'].items():
        print(f"  {turma}: {cursos}")

    print("\n👨‍🏫 DSD (Professores e Cursos):")
    for prof, cursos in dados['dsd'].items():
        print(f"  {prof}: {cursos}")

    print("\n⏰ TR (Restrições de Tempo):")
    for prof, slots in dados['tr'].items():
        print(f"  {prof}: slots {slots}")

    print("\n🚪 RR (Restrições de Sala):")
    for curso, sala in dados['rr'].items():
        print(f"  {curso}: {sala}")

    print("\n💻 OC (Aulas Online):")
    for curso, indice in dados['oc'].items():
        print(f"  {curso}: aula {indice}")

    print("\n📝 OLW (Cursos com 1 aula/semana):")
    print(f"  {dados['olw']}")

✅ Dataset carregado com sucesso!

📊 CONTEÚDO PARSEADO:

🎓 HEAD (Configurações):
{}

📚 CC (Classes e Cursos):
  t01: ['UC11', 'UC12', 'UC13', 'UC14', 'UC15']
  t02: ['UC21', 'UC22', 'UC23', 'UC24', 'UC25']
  t03: ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']

👨‍🏫 DSD (Professores e Cursos):
  jo: ['UC11', 'UC21', 'UC22', 'UC31']
  mike: ['UC12', 'UC23', 'UC32']
  rob: ['UC13', 'UC14', 'UC24', 'UC33']
  sue: ['UC15', 'UC25', 'UC34', 'UC35']

⏰ TR (Restrições de Tempo):
  mike: slots [13, 14, 15, 16, 17, 18, 19, 20]
  rob: slots [1, 2, 3, 4]
  sue: slots [9, 10, 11, 12, 17, 18, 19, 20]

🚪 RR (Restrições de Sala):
  UC14: Lab01
  UC22: Lab01

💻 OC (Aulas Online):
  UC21: aula 2
  UC31: aula 2

📝 OLW (Cursos com 1 aula/semana):
  []


In [6]:
# ============================
# 1) VARIÁVEIS (duas por UC)
# ============================
lesson_vars = []                      # ex: "t01_UC11_L1"
lesson_meta = {}                      # var -> {"class":..., "course":..., "lesson":1|2}
for turma, ucs in dados['cc'].items():
    for uc in ucs:
        for i in (1, 2):              # cada UC tem 2 aulas/semana
            var = f"{turma}_{uc}_L{i}"
            lesson_vars.append(var)
            lesson_meta[var] = {"class": turma, "course": uc, "lesson": i}

print(f"Total de variáveis criadas: {len(lesson_vars)}")


Total de variáveis criadas: 30


In [7]:
from constraint import Problem
from collections import Counter

# Criar o problema
problem = Problem()

# Adicionar todas as variáveis ao problema (domínio: blocos 1..20)
for v in lesson_vars:
    problem.addVariable(v, range(1, 21))  # 20 blocos possíveis

# Função auxiliar para descobrir o "dia" de um bloco
def dia_do_bloco(slot):
    return ((slot - 1) // 4) + 1   # blocos 1–4 = dia 1, 5–8 = dia 2, etc.

# Restrição: máximo 3 aulas por dia por turma
def max_4_por_dia(*slots):
    dias = [dia_do_bloco(s) for s in slots]  # converte blocos em dias
    contagem = Counter(dias)
    return all(qtd <= 4 for qtd in contagem.values())

# Aplicar a restrição a cada turma
for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    problem.addConstraint(max_4_por_dia, turma_vars)


# ===============================================
# RESTRIÇÃO: Respeitar disponibilidade dos docentes
# ===============================================

# Vamos criar um mapa curso -> professor
curso_professor = {}
for prof, cursos in dados['dsd'].items():
    for c in cursos:
        curso_professor[c] = prof

# Agora aplicar as restrições
for prof, indisponiveis in dados['tr'].items():
    cursos_prof = [c for c, p in curso_professor.items() if p == prof]
    for turma, ucs in dados['cc'].items():
        for uc in ucs:
            if uc in cursos_prof:
                for i in (1, 2):
                    var = f"{turma}_{uc}_L{i}"
                    bloqueados = set(indisponiveis)  # <– captura local
                    problem.addConstraint(
                        lambda slot, bloqueados=bloqueados: slot not in bloqueados,
                        (var,)
                    )


# ===============================================
# RESTRIÇÃO: Aulas online (máx. 3 por dia)
# ===============================================

# Identificar as variáveis que são online
online_vars = []
for curso, li in dados['oc'].items():   # ex: UC21 -> aula 2
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            var = f"{turma}_{curso}_L{li}"
            online_vars.append(var)

print("Variáveis de aulas online:", online_vars)

# Função de restrição: máximo 3 online no mesmo dia
def max_3_online(*slots):
    dias = [dia_do_bloco(s) for s in slots]
    contagem = Counter(dias)
    return all(qtd <= 3 for qtd in contagem.values())

# Aplicar a restrição (somente se existirem aulas online)
if online_vars:
    problem.addConstraint(max_3_online, online_vars)

# ===============================================
# RESTRIÇÃO: Salas específicas para certas UCs
# ===============================================

# Criar um dicionário var -> sala fixa
salas_fixas = {}
for curso, sala in dados['rr'].items():  # ex: UC14 -> Lab01
    for turma, ucs in dados['cc'].items():
        if curso in ucs:
            for i in (1, 2):  # cada UC tem 2 aulas (L1, L2)
                var = f"{turma}_{curso}_L{i}"
                salas_fixas[var] = sala

print("Salas fixas atribuídas:", salas_fixas)

# ===============================================
# 🧠 RESTRIÇÕES DE CONFLITO — Professores, turmas e salas
# ===============================================


# 1️⃣ Professores não podem dar duas aulas ao mesmo tempo (par a par)
for prof, cursos in dados['dsd'].items():
    prof_vars = [v for v in lesson_vars if any(uc in v for uc in cursos)]
    for i in range(len(prof_vars)):
        for j in range(i + 1, len(prof_vars)):
            problem.addConstraint(lambda a, b: a != b, (prof_vars[i], prof_vars[j]))

# 2️⃣ Cada turma só pode ter uma aula por bloco (par a par)
for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    for i in range(len(turma_vars)):
        for j in range(i + 1, len(turma_vars)):
            problem.addConstraint(lambda a, b: a != b, (turma_vars[i], turma_vars[j]))

# 3️⃣ Cada sala fixa (ex: Lab01) só pode ter uma UC por bloco
for sala in set(salas_fixas.values()):
    sala_vars = [v for v, s in salas_fixas.items() if s == sala]
    if sala_vars:
        problem.addConstraint(lambda *slots: len(slots) == len(set(slots)), sala_vars)


# ===============================================
# 💡 NOVA RESTRIÇÃO: Pelo menos 1 aula na segunda-feira por turma
# ===============================================

def pelo_menos_uma_na_segunda(*slots):
    dias = [dia_do_bloco(s) for s in slots]
    return 1 in dias  # dia 1 = segunda-feira

for turma in dados['cc']:
    turma_vars = [v for v in lesson_vars if v.startswith(turma + "_")]
    problem.addConstraint(pelo_menos_uma_na_segunda, turma_vars)



# ===============================================
# 🚀 TENTAR OBTER UMA SOLUÇÃO E MOSTRAR HORÁRIO
# ===============================================


solution = next(problem.getSolutionIter(), None)

if not solution:
    print("❌ Nenhuma solução viável encontrada — verifica as restrições (pode haver conflito).")
else:
    print("✅ Solução viável encontrada!\n")
    print("==================================================")
    print("📅 HORÁRIO GERAL (por turma, UC, professor e sala)")
    print("==================================================\n")

    def bloco_para_horario(slot):
        dia_idx = (slot - 1) // 4
        bloco_dia = (slot - 1) % 4 + 1
        dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
        horas = {1: "09h–11h", 2: "11h–13h", 3: "14h–16h", 4: "16h–18h"}
        return dias_semana[dia_idx], horas[bloco_dia]

    curso_professor = {}
    for prof, cursos in dados['dsd'].items():
        for c in cursos:
            curso_professor[c] = prof

    # Criar um conjunto com as aulas online (turma_UC_Lx)
    online_set = set()
    for curso, li in dados['oc'].items():
        for turma, ucs in dados['cc'].items():
            if curso in ucs:
                var = f"{turma}_{curso}_L{li}"
                online_set.add(var)

    # Agrupar o resultado por turma
    for turma in sorted(dados['cc'].keys()):
        print(f"\n🧩 Turma {turma}:")
        print("-" * 60)
        vars_turma = [v for v in sorted(solution.keys()) if v.startswith(turma + "_")]

        horario = {}
        for v in vars_turma:
            uc = v.split("_")[1]
            slot = solution[v]
            dia, hora = bloco_para_horario(slot)
            prof = curso_professor.get(uc, "—")
            sala = salas_fixas.get(v, "SalaGenérica")

            # Marcar se é online
            tipo = "💻 Online" if v in online_set else sala
            horario.setdefault(dia, []).append((hora, uc, prof, tipo))

        for dia in ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]:
            if dia in horario:
                print(f"\n  📆 {dia}:")
                for hora, uc, prof, tipo in sorted(horario[dia]):
                    print(f"    {hora} → {uc}  ({tipo}) — Prof. {prof}")


Variáveis de aulas online: ['t02_UC21_L2', 't03_UC31_L2']
Salas fixas atribuídas: {'t01_UC14_L1': 'Lab01', 't01_UC14_L2': 'Lab01', 't02_UC22_L1': 'Lab01', 't02_UC22_L2': 'Lab01'}
✅ Solução viável encontrada!

📅 HORÁRIO GERAL (por turma, UC, professor e sala)


🧩 Turma t01:
------------------------------------------------------------

  📆 Segunda:
    14h–16h → UC12  (SalaGenérica) — Prof. mike

  📆 Quarta:
    11h–13h → UC12  (SalaGenérica) — Prof. mike
    14h–16h → UC11  (SalaGenérica) — Prof. jo
    16h–18h → UC11  (SalaGenérica) — Prof. jo

  📆 Quinta:
    09h–11h → UC13  (SalaGenérica) — Prof. rob
    11h–13h → UC13  (SalaGenérica) — Prof. rob
    14h–16h → UC15  (SalaGenérica) — Prof. sue
    16h–18h → UC15  (SalaGenérica) — Prof. sue

  📆 Sexta:
    14h–16h → UC14  (Lab01) — Prof. rob
    16h–18h → UC14  (Lab01) — Prof. rob

🧩 Turma t02:
------------------------------------------------------------

  📆 Segunda:
    11h–13h → UC23  (SalaGenérica) — Prof. mike

  📆 Quarta:
    11h